In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns

In [13]:
data = pd.read_csv("train.csv") 

In [14]:
test_data = pd.read_csv("test.csv")

In [16]:
data.Cabin = data.Cabin.apply(lambda x: 1 if x!=np.nan else 0)

In [17]:
data.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,1,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,1,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,1,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,1,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,1,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,1,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,1,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,1,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,1,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,1,C


In [6]:
sex_mean_age = data.groupby("Sex").aggregate({"Age":"mean"})

In [7]:
sex_mean_age = sex_mean_age.transpose()

In [11]:
data["Sex"] = data["Sex"].apply(lambda x : int(x))

In [9]:
data.loc[(data["Sex"] == True)&(data["Age"].isna()),"Age"] = round(sex_mean_age.male.iloc[0])

In [12]:
data.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",1,22.0,1,0,A/5 21171,7.2500,0,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",0,26.0,0,0,STON/O2. 3101282,7.9250,0,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",1,35.0,0,0,373450,8.0500,0,S
5,6,0,3,"Moran, Mr. James",1,31.0,0,0,330877,8.4583,0,Q
6,7,0,1,"McCarthy, Mr. Timothy J",1,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",1,2.0,3,1,349909,21.0750,0,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",0,27.0,0,2,347742,11.1333,0,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",0,14.0,1,0,237736,30.0708,0,C


In [11]:
#data.Embarked = data["Embarked"].fillna('O').map({'S':3,'C':2,'Q':1,"O":0})

In [12]:
#data.Cabin.unique()

# Auto data predprocessing

In [29]:
from random import randint
from sklearn.model_selection import train_test_split, GridSearchCV


def process_data(data:pd.DataFrame)->tuple[pd.DataFrame]:
    df = data.copy()
    df.Cabin = df.Cabin.apply(lambda x: 1 if x==np.nan else 0)
    df.Sex = df.Sex.apply(lambda x : 1 if x=="male" else 0)
    df = df.rename(columns = {"Cabin":"HasCabin"})
    df['Embarked'] = df['Embarked'].fillna(df['Embarked'].mode()[0])
    sex_mean_age = data.groupby("Sex").aggregate({"Age":"mean"})
    sex_mean_age = sex_mean_age.transpose()
    df.loc[(df["Sex"] == True)&(df["Age"].isna()),"Age"] = round(sex_mean_age.male.iloc[0])
    X = df.drop(["Survived"],axis=1)
    y = df.Survived
    return X,y
    

X,y = process_data(data)
#test_data

(     PassengerId  Pclass                                               Name  \
 0              0       0                                                  0   
 1              2       1  Cumings, Mrs. John Bradley (Florence Briggs Th...   
 2              0       0                                                  0   
 3              4       1       Futrelle, Mrs. Jacques Heath (Lily May Peel)   
 4              0       0                                                  0   
 ..           ...     ...                                                ...   
 886            0       0                                                  0   
 887          888       1                       Graham, Miss. Margaret Edith   
 888            0       0                                                  0   
 889          890       1                              Behr, Mr. Karl Howell   
 890            0       0                                                  0   
 
      Sex   Age  SibSp  Parch    Ticke

# Trees

In [19]:
X = data.drop(["Name","Ticket"],axis=1)
X_train, X_val, y_train, y_val = train_test_split(X,y,test_size = 0.25)

In [20]:
X.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,1,0,3,male,22.0,1,0,7.2500,NaN,S
1,2,1,1,female,38.0,1,0,71.2833,C85,C
2,3,1,3,female,26.0,0,0,7.9250,NaN,S
3,4,1,1,female,35.0,1,0,53.1000,C123,S
4,5,0,3,male,35.0,0,0,8.0500,NaN,S


In [22]:

from sklearn.tree import DecisionTreeClassifier

In [23]:
tree = DecisionTreeClassifier()

In [24]:
param_grid={"criterion":["gini","entropy"],"max_depth":range(1,10)}
search = GridSearchCV(tree, param_grid=param_grid, cv=5, scoring='f1')

In [25]:
search.fit(X_train,y_train)

ValueError: 
All the 90 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
18 fits failed with the following error:
Traceback (most recent call last):
  File "/home/vadimpp/.local/lib/python3.12/site-packages/sklearn/model_selection/_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/vadimpp/.local/lib/python3.12/site-packages/sklearn/base.py", line 1389, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/vadimpp/.local/lib/python3.12/site-packages/sklearn/tree/_classes.py", line 1024, in fit
    super()._fit(
  File "/home/vadimpp/.local/lib/python3.12/site-packages/sklearn/tree/_classes.py", line 252, in _fit
    X, y = validate_data(
           ^^^^^^^^^^^

In [45]:
best_tree = search.best_estimator_

In [46]:
best_tree.score(X_train,y_train)

0.8226711560044894

In [49]:
from sklearn.metrics import f1_score

In [50]:
f1_score(y_train,best_tree.predict(X_train))

0.7546583850931677

In [189]:
pred = best_tree.predict(X_train)

In [194]:
res = pd.DataFrame({"PassengerID":X_train.PassengerId,"Survived":pred})

In [195]:
res.to_csv("subm.csv",index=False)